In [1]:
%%html
<style type='text/css'>
.CodeMirror{
    font-family: Inconsolata;
    font-size: 13px;
}

div.output_area pre {
    font-family: Inconsolata;
    font-size: 11px;
}

div.text_cell_render {
    font-family: Inconsolata;
    font-size: 13px;
}
</style>

In [2]:
import pandas as pd
import os, shutil
import numpy as np
import matplotlib
import sys; sys.path.append('..')
import pickle
import requests
import torch

from queue import Queue
from tqdm.notebook import tqdm, tqdm_notebook
from collections import defaultdict

path_ddi_dataset = r"/data/data2/041/datasets/DDI"
path_iii_dataset = r"/data/data2/041/datasets/mimic-iii-clinical-database-1.4"

## TWOSIDES

In [55]:
df_twosides = pd.read_csv(os.path.join(path_ddi_dataset, "TWOSIDES.csv.gz"))
df_twosides["drug_1_rxnorn_id"] = df_twosides["drug_1_rxnorn_id"].astype("string")

df_twosides.info()

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (0,2,4,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42920391 entries, 0 to 42920390
Data columns (total 13 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   drug_1_rxnorn_id          string
 1   drug_1_concept_name       object
 2   drug_2_rxnorm_id          object
 3   drug_2_concept_name       object
 4   condition_meddra_id       object
 5   condition_concept_name    object
 6   A                         object
 7   B                         object
 8   C                         object
 9   D                         object
 10  PRR                       object
 11  PRR_error                 object
 12  mean_reporting_frequency  object
dtypes: object(12), string(1)
memory usage: 4.2+ GB


In [56]:
df_twosides.head()

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia,7,149,24,1536,2.91667,0.421275,0.0448718
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia,1,13,2,138,5.0,1.19224,0.0714286
2,221147,POLYETHYLENE GLYCOL 3350,5521,Hydroxychloroquine,10003239,Arthralgia,6,103,20,1070,3.0,0.454505,0.0550459
3,10324,Tamoxifen,8640,Prednisone,10012735,Diarrhoea,18,123,35,1375,5.14286,0.276271,0.12766
4,10355,Temazepam,136411,sildenafil,10012735,Diarrhoea,2,154,37,1523,0.540541,0.721093,0.0128205


In [64]:
df_twosides.tail()

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
42920386,6142,Ketoprofen,88249,montelukast,10050954,Activities of daily living impaired,1,51,1,519,10.0,1.40671,0.019231
42920387,6142,Ketoprofen,88249,montelukast,10052302,Impaired work ability,1,51,1,519,10.0,1.40671,0.019231
42920388,6142,Ketoprofen,88249,montelukast,10053762,Off label use,3,49,13,507,2.30769,0.623781,0.057692
42920389,6142,Ketoprofen,88249,montelukast,10020772,Hypertension,1,51,4,516,2.5,1.10853,0.019231
42920390,6142,Ketoprofen,88249,montelukast,10070863,Toxicity to various agents,1,51,3,517,3.33333,1.1455,0.019231


---

# MAPPING

1. NDC to RXCUI (done in `preprocess_drug.ipynb`)
2. RXCUI to ATC4
3. ATC4 to CID

## RXCUI2ATC4

In [138]:
df_ndc2atc = pd.read_csv(os.path.join(path_ddi_dataset, "ndc2atc_level4.csv"))
df_ndc2atc.drop(columns=['YEAR','MONTH','NDC'], inplace=True)
df_ndc2atc

,RXCUI,ATC4
0,853004,C01BA
1,1551300,A10BJ
2,1551300,A10BJ
3,1551306,A10BJ
4,1551306,A10BJ
...,...,...
318252,543484,D07AC
318253,543484,C05AA
318254,543484,D07AC
318255,1482689,D01AC


In [139]:
df_ndc2atc.RXCUI.value_counts()

310965     5585
204602     4709
581660     3630
312172     3365
106258     2457
           ... 
1242136       1
1053146       1
211199        1
860898        1
352213        1
Name: RXCUI, Length: 11478, dtype: int64

In [140]:
df_ndc2atc.drop_duplicates(subset=['RXCUI'], inplace=True)
df_ndc2atc.RXCUI.value_counts()

311296     1
903884     1
898689     1
1094323    1
311989     1
          ..
1736053    1
1789192    1
830839     1
152952     1
1116159    1
Name: RXCUI, Length: 11478, dtype: int64

In [188]:
df_drug_ndc_feat = pd.read_csv(os.path.join(path_iii_dataset, "DRUGS_NDC_FEAT.csv.gz"))
df_drug_ndc_feat.drop(columns=['Unnamed: 0'], inplace=True)
df_drug_ndc_feat.drop(columns=[
    'DRUG_TYPE_MAIN_Proportion', 
    'DRUG_TYPE_BASE_Proportion', 
    'DRUG_TYPE_ADDITIVE_Proportion', 
    'FORM_UNIT_DISP_Freq_1', 
    'FORM_UNIT_DISP_Freq_2', 
    'FORM_UNIT_DISP_Freq_3', 
    'FORM_UNIT_DISP_Freq_4', 
    'FORM_UNIT_DISP_Freq_5'], inplace=True)
df_drug_ndc_feat.sort_values(by="NDC", inplace=True)
df_drug_ndc_feat['idx'] = range(len(df_drug_ndc_feat))
df_drug_ndc_feat.head()

,NDC,rxnorm_id,idx
4121,1.0,NaN,0
4122,2.0,NaN,1
4123,3.0,NaN,2
4124,4.0,NaN,3
4125,5.0,NaN,4


In [223]:
df_drug_ndc_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4294 entries, 4121 to 4120
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NDC        4294 non-null   float64
 1   rxnorm_id  4101 non-null   float64
 2   idx        4294 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 134.2 KB


In [222]:
df_drug_ndc_feat_ndc_not_real = df_drug_ndc_feat[df_drug_ndc_feat.NDC < 1000].copy()
df_drug_ndc_feat_ndc_not_real.rename(columns={'rxnorm_id': 'RXCUI'}, inplace=True)
df_drug_ndc_feat_ndc_not_real

,NDC,RXCUI,idx
4121,1.0,NaN,0
4122,2.0,NaN,1
4123,3.0,NaN,2
4124,4.0,NaN,3
4125,5.0,NaN,4
...,...,...,...
4289,170.0,NaN,168
4290,171.0,NaN,169
4291,172.0,NaN,170
4292,173.0,NaN,171


In [220]:
temp_df = df_drug_ndc_feat[df_drug_ndc_feat.NDC > 1000]
df_drug_ndc_feat_ndc_real_rxcui_not_exist = temp_df[temp_df.rxnorm_id.isnull()].copy()
df_drug_ndc_feat_ndc_real_rxcui_not_exist.rename(columns={'rxnorm_id': 'RXCUI'}, inplace=True)
df_drug_ndc_feat_ndc_real_rxcui_not_exist

,NDC,RXCUI,idx
416,2.606320e+07,NaN,589
1351,1.320001e+08,NaN,1524
1798,3.380404e+08,NaN,1971
2569,8.290002e+09,NaN,2742
2576,8.881200e+09,NaN,2749
2639,1.093900e+10,NaN,2812
2666,1.253901e+10,NaN,2839
2677,1.512702e+10,NaN,2850
2682,1.656301e+10,NaN,2855
2684,1.683709e+10,NaN,2857


In [190]:
df_drug_ndc_feat_ndc_real = df_drug_ndc_feat[df_drug_ndc_feat.NDC > 1000].copy()
df_drug_ndc_feat_ndc_real.dropna(inplace=True)
df_drug_ndc_feat_ndc_real

,NDC,rxnorm_id,idx
0,2.050101e+06,207882.0,173
1,2.140701e+06,853004.0,174
2,2.144401e+06,208815.0,175
3,2.202402e+06,313009.0,176
4,2.322930e+06,352320.0,177
...,...,...,...
4114,7.431200e+10,309604.0,4287
4115,7.431200e+10,313896.0,4288
4116,7.811200e+10,1101849.0,4289
4117,7.951105e+10,669319.0,4290


In [191]:
df_drug_ndc_feat_ndc_real.rename(columns={'rxnorm_id': 'RXCUI'}, inplace=True)
df_drug_ndc_feat_ndc_real

,NDC,RXCUI,idx
0,2.050101e+06,207882.0,173
1,2.140701e+06,853004.0,174
2,2.144401e+06,208815.0,175
3,2.202402e+06,313009.0,176
4,2.322930e+06,352320.0,177
...,...,...,...
4114,7.431200e+10,309604.0,4287
4115,7.431200e+10,313896.0,4288
4116,7.811200e+10,1101849.0,4289
4117,7.951105e+10,669319.0,4290


In [192]:
df_drug_ndc_feat_ndc_real = df_drug_ndc_feat_ndc_real.merge(df_ndc2atc, how='left', on=["RXCUI"])
df_drug_ndc_feat_ndc_real

,NDC,RXCUI,idx,ATC4
0,2.050101e+06,207882.0,173,NaN
1,2.140701e+06,853004.0,174,C01BA
2,2.144401e+06,208815.0,175,NaN
3,2.202402e+06,313009.0,176,A12CA
4,2.322930e+06,352320.0,177,N06BA
...,...,...,...,...
4096,7.431200e+10,309604.0,4287,NaN
4097,7.431200e+10,313896.0,4288,NaN
4098,7.811200e+10,1101849.0,4289,NaN
4099,7.951105e+10,669319.0,4290,NaN


In [ ]:
# df_drug_ndc_feat_ndc_real['ATC4'] = df_drug_ndc_feat_ndc_real['ATC4'].map(lambda x: x[:4])
# df_drug_ndc_feat_ndc_real

In [194]:
df_drug_ndc_feat_ndc_real_atc4_exist = df_drug_ndc_feat_ndc_real[df_drug_ndc_feat_ndc_real.ATC4.notnull()]
df_drug_ndc_feat_ndc_real_atc4_exist

,NDC,RXCUI,idx,ATC4
1,2.140701e+06,853004.0,174,C01BA
3,2.202402e+06,313009.0,176,A12CA
4,2.322930e+06,352320.0,177,N06BA
5,2.323560e+06,596928.0,178,N06AX
6,2.324001e+06,596932.0,179,N06AX
...,...,...,...,...
4083,6.854603e+10,1111642.0,4273,L03AX
4086,6.877401e+10,197516.0,4276,J01FA
4088,6.885000e+10,995605.0,4278,J04AK
4089,6.885000e+10,995609.0,4279,J04AK


In [195]:
df_drug_ndc_feat_ndc_real_atc4_exist.ATC4.value_counts()

A01AC    61
N02AA    60
N03AX    47
C09AA    40
N05AX    38
         ..
D10AE     1
C01DX     1
A10BF     1
N05AG     1
M01AE     1
Name: ATC4, Length: 346, dtype: int64

In [196]:
df_drug_ndc_feat_ndc_real_ATC4_unique = df_drug_ndc_feat_ndc_real_atc4_exist.drop_duplicates(subset=['ATC4'])
df_drug_ndc_feat_ndc_real_ATC4_unique

,NDC,RXCUI,idx,ATC4
1,2.140701e+06,853004.0,174,C01BA
3,2.202402e+06,313009.0,176,A12CA
4,2.322930e+06,352320.0,177,N06BA
5,2.323560e+06,596928.0,178,N06AX
9,2.411230e+06,212405.0,182,N05AH
...,...,...,...,...
3937,6.476402e+10,617768.0,4126,A06AX
3947,6.519700e+10,861404.0,4136,C04AX
3967,6.584702e+10,883708.0,4156,C01DX
3988,6.658204e+10,352304.0,4177,C10AX


In [148]:
list_atc4_exist = list(df_drug_ndc_feat_ndc_real_ATC4_unique.ATC4.unique())
list_atc4_exist

['C01BA',
 'A12CA',
 'N06BA',
 'N06AX',
 'N05AH',
 'G03XC',
 'B01AC',
 'A10AB',
 'H05AA',
 'A01AC',
 'L01XE',
 'J01CE',
 'L01XX',
 'J05AF',
 'L04AA',
 'J01DF',
 'J01GB',
 'J05AE',
 'C10AA',
 'J05AB',
 'N03AE',
 'D01AE',
 'A11CC',
 'M05BA',
 'C03CA',
 'L03AB',
 'J05AX',
 'J05AH',
 'L01BC',
 'N04BX',
 'S01EC',
 'H03BA',
 'A01AB',
 'D11AX',
 'B02BA',
 'C03CC',
 'R03DC',
 'C01EB',
 'N02CC',
 'A10BH',
 'A04AD',
 'C09CA',
 'A02BA',
 'N04AC',
 'J01DH',
 'C03AA',
 'J02AX',
 'L01AA',
 'B01AX',
 'N05AB',
 'C03DB',
 'C07AG',
 'L01BB',
 'B01AE',
 'N06AF',
 'N04BC',
 'H03AA',
 'P02CA',
 'D04AA',
 'N02AA',
 'N05CA',
 'N05BA',
 'A02BC',
 'N05CF',
 'A06AH',
 'C07AB',
 'L01XC',
 'J01AA',
 'D07AA',
 'G03AC',
 'D06AX',
 'C10AC',
 'A10BB',
 'G03BA',
 'D10AF',
 'G02AD',
 'C01EA',
 'J05AG',
 'C01BD',
 'G04BD',
 'L01DB',
 'J01XX',
 'L02BG',
 'A07EC',
 'B02AA',
 'J04AB',
 'S01EE',
 'L01AD',
 'L01XA',
 'R05CB',
 'J01CF',
 'J01CA',
 'A06AA',
 'M01AB',
 'P03AC',
 'L04AD',
 'C01CA',
 'L01XD',
 'P01BA',
 'C03DA',


## ATC2CID

In [205]:
cid_atc = os.path.join(path_ddi_dataset, "drug-atc.csv")
cid2atc_dic = {}
with open(cid_atc, 'r') as f:
    for line in f:
        line_ls = line[:-1].split(',')
        cid = line_ls[0]
        atcs = [act[:5] for act in line_ls[1:]]
        cid2atc_dic[cid] = atcs

cid2atc_dic

{'CID000003015': ['G03HA'],
 'CID000004011': ['N06AA'],
 'CID000071273': ['N01BB'],
 'CID000062816': ['C10AC'],
 'CID000052421': ['D07AC', 'V01AA', 'V01AA'],
 'CID000056339': ['N03AB'],
 'CID000077992': ['N02CC'],
 'CID000005300': ['L01AD'],
 'CID000222786': ['H02AB', 'S01BA'],
 'CID000071301': ['C07AB'],
 'CID000004870': ['C03AA'],
 'CID000004873': ['A12BA', 'B05XA'],
 'CID000002142': ['D06AX', 'J01GB', 'S01AA'],
 'CID000002141': ['V03AF'],
 'CID000131536': ['J05AE'],
 'CID000001775': ['N03AB', 'N03AB', 'N03AB'],
 'CID000003305': ['M05BA', 'M05BB'],
 'CID000005454': ['N05AF'],
 'CID000003308': ['M01AB'],
 'CID000004909': ['D08AC', 'S03AA', 'S01AX', 'R01AX', 'R02AA', 'N03AA'],
 'CID000004908': ['P01BA'],
 'CID000002099': ['A03AE'],
 'CID000054688': ['J01FA'],
 'CID000002092': ['G04CA'],
 'CID000005503': ['A10BB'],
 'CID000004536': ['G03AC'],
 'CID006918453': ['M01CB'],
 'CID000003706': ['J05AE'],
 'CID000005408': ['G03BA', 'G03BA', 'G03EK'],
 'CID000004497': ['C08CA'],
 'CID000004493':

In [210]:
act2cid = {}
for curr_atc in tqdm(list_atc4_exist):
    
    unique_cid = []
    for cid, list_atc in cid2atc_dic.items():
        if curr_atc in list_atc:
            unique_cid.append(cid)
    if len(unique_cid) != 0:
        act2cid[curr_atc] = unique_cid  # each exist act4 code may have multiple cid codes
    
act2cid

  0%|          | 0/346 [00:00<?, ?it/s]

{'C01BA': ['CID000004914', 'CID000004913', 'CID000003114', 'CID000001065'],
 'N06BA': ['CID000054841',
  'CID000003007',
  'CID000004723',
  'CID000004158',
  'CID000004236',
  'CID000001206'],
 'N06AX': ['CID000005533',
  'CID000004449',
  'CID000060835',
  'CID000004205',
  'CID000125017',
  'CID000001148'],
 'N05AH': ['CID000002818', 'CID000005002', 'CID000003964'],
 'G03XC': ['CID000005035'],
 'B01AC': ['CID000003108',
  'CID000002806',
  'CID000002244',
  'CID000000159',
  'CID005311181',
  'CID000005486',
  'CID000005472',
  'CID000054786'],
 'A01AC': ['CID000003003', 'CID000005544'],
 'L01XE': ['CID000176870',
  'CID005329102',
  'CID000005291',
  'CID000123631',
  'CID003062316'],
 'J01CE': ['CID000004914', 'CID000004730', 'CID000002349'],
 'L01XX': ['CID000018140',
  'CID000005515',
  'CID000002182',
  'CID000005538',
  'CID000093860',
  'CID000003657',
  'CID000002662',
  'CID000003750',
  'CID000002123',
  'CID000004211',
  'CID000082146',
  'CID000014888',
  'CID000002179']

In [207]:
df_drug_ndc_feat_ndc_real['list_cid'] = df_drug_ndc_feat_ndc_real['ATC4'].map(act2cid)
df_drug_ndc_feat_ndc_real

,NDC,RXCUI,idx,ATC4,list_cid
0,2.050101e+06,207882.0,173,NaN,NaN
1,2.140701e+06,853004.0,174,C01BA,"[CID000004914, CID000004913, CID000003114, CID..."
2,2.144401e+06,208815.0,175,NaN,NaN
3,2.202402e+06,313009.0,176,A12CA,NaN
4,2.322930e+06,352320.0,177,N06BA,"[CID000054841, CID000003007, CID000004723, CID..."
...,...,...,...,...,...
4096,7.431200e+10,309604.0,4287,NaN,NaN
4097,7.431200e+10,313896.0,4288,NaN,NaN
4098,7.811200e+10,1101849.0,4289,NaN,NaN
4099,7.951105e+10,669319.0,4290,NaN,NaN


In [224]:
df_map_of_idx4ndc_rxcui_atc4_cids = pd.concat([
    df_drug_ndc_feat_ndc_real_rxcui_not_exist, 
    df_drug_ndc_feat_ndc_real,
    df_drug_ndc_feat_ndc_not_real
])
df_map_of_idx4ndc_rxcui_atc4_cids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4294 entries, 416 to 4293
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NDC       4294 non-null   float64
 1   RXCUI     4101 non-null   float64
 2   idx       4294 non-null   int64  
 3   ATC4      2885 non-null   object 
 4   list_cid  2629 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 201.3+ KB


In [232]:
# df_map_of_idx4ndc_rxcui_atc4_cids.to_csv(os.path.join(path_ddi_dataset, "MAP_IDX4NDC_RXCUI_ATC4_CIDS.csv"))

## NDC index for whose CID exist

In [229]:
df_map_of_idx4ndc_rxcui_atc4_cids[df_map_of_idx4ndc_rxcui_atc4_cids.list_cid.notnull()]

,NDC,RXCUI,idx,ATC4,list_cid
1,2.140701e+06,853004.0,174,C01BA,"[CID000004914, CID000004913, CID000003114, CID..."
4,2.322930e+06,352320.0,177,N06BA,"[CID000054841, CID000003007, CID000004723, CID..."
5,2.323560e+06,596928.0,178,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
6,2.324001e+06,596932.0,179,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
7,2.324030e+06,596932.0,180,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
...,...,...,...,...,...
4082,6.854601e+10,637218.0,4272,N04BD,[CID000122316]
4086,6.877401e+10,197516.0,4276,J01FA,"[CID000054688, CID003002190, CID000002269, CID..."
4088,6.885000e+10,995605.0,4278,J04AK,"[CID000003279, CID000001046]"
4089,6.885000e+10,995609.0,4279,J04AK,"[CID000003279, CID000001046]"


In [230]:
list_ndc_idx_can_calc_ddi = list(
    df_map_of_idx4ndc_rxcui_atc4_cids[df_map_of_idx4ndc_rxcui_atc4_cids.list_cid.notnull()]\
    .idx.unique()
)
list_ndc_idx_can_calc_ddi

# can be used for slide

[174,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 195,
 205,
 216,
 220,
 221,
 225,
 227,
 228,
 230,
 231,
 232,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 244,
 245,
 248,
 250,
 251,
 252,
 253,
 254,
 257,
 258,
 262,
 264,
 270,
 271,
 272,
 273,
 280,
 281,
 282,
 283,
 284,
 285,
 293,
 297,
 299,
 303,
 304,
 308,
 309,
 311,
 312,
 313,
 316,
 317,
 318,
 321,
 322,
 325,
 326,
 327,
 328,
 329,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 342,
 344,
 345,
 346,
 347,
 351,
 357,
 358,
 359,
 368,
 375,
 376,
 377,
 378,
 382,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 397,
 398,
 399,
 402,
 403,
 404,
 407,
 408,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 438,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 462,
 463,
 464,
 465,
 466,
 468,
 470,
 471,
 472

In [231]:
len(list_ndc_idx_can_calc_ddi)

2629

# adjacency matrix of DDI

In [211]:
unique_exist_cid = []
for atc, list_cid in act2cid.items():
    unique_exist_cid.extend(list_cid)

unique_exist_cid = list(set(unique_exist_cid))  # make sure the uniqueness of cid
unique_exist_cid

['CID005311027',
 'CID000003954',
 'CID000060871',
 'CID000004914',
 'CID000005064',
 'CID000004724',
 'CID000003393',
 'CID000004594',
 'CID000004138',
 'CID000003961',
 'CID000005718',
 'CID000004062',
 'CID000005408',
 'CID000083786',
 'CID000004411',
 'CID000038904',
 'CID000125017',
 'CID000001206',
 'CID000005544',
 'CID000005403',
 'CID000005076',
 'CID000002130',
 'CID000002370',
 'CID000003964',
 'CID000051263',
 'CID000028112',
 'CID000156419',
 'CID000004536',
 'CID000004909',
 'CID000003417',
 'CID000002375',
 'CID000003648',
 'CID000006691',
 'CID000002905',
 'CID000006256',
 'CID001349907',
 'CID000034312',
 'CID000003114',
 'CID000004485',
 'CID000060835',
 'CID000003690',
 'CID000003366',
 'CID000003405',
 'CID000004915',
 'CID000005486',
 'CID000477468',
 'CID000005978',
 'CID000001978',
 'CID000005212',
 'CID000004197',
 'CID000060198',
 'CID000003040',
 'CID000213039',
 'CID000005040',
 'CID000005391',
 'CID000002727',
 'CID000004834',
 'CID000000951',
 'CID000005359

In [212]:
len(unique_exist_cid)

675

In [213]:
def cmp_cid(cid: str):
    return int(cid[3:])
    
unique_exist_cid.sort(key=cmp_cid)
unique_exist_cid

['CID000000085',
 'CID000000137',
 'CID000000143',
 'CID000000158',
 'CID000000159',
 'CID000000187',
 'CID000000191',
 'CID000000298',
 'CID000000401',
 'CID000000450',
 'CID000000453',
 'CID000000581',
 'CID000000596',
 'CID000000681',
 'CID000000727',
 'CID000000750',
 'CID000000772',
 'CID000000815',
 'CID000000838',
 'CID000000853',
 'CID000000861',
 'CID000000888',
 'CID000000942',
 'CID000000951',
 'CID000001046',
 'CID000001065',
 'CID000001148',
 'CID000001206',
 'CID000001546',
 'CID000001775',
 'CID000001935',
 'CID000001971',
 'CID000001972',
 'CID000001978',
 'CID000001986',
 'CID000002021',
 'CID000002022',
 'CID000002082',
 'CID000002083',
 'CID000002088',
 'CID000002092',
 'CID000002118',
 'CID000002123',
 'CID000002130',
 'CID000002131',
 'CID000002133',
 'CID000002141',
 'CID000002142',
 'CID000002145',
 'CID000002153',
 'CID000002156',
 'CID000002160',
 'CID000002162',
 'CID000002170',
 'CID000002171',
 'CID000002173',
 'CID000002177',
 'CID000002179',
 'CID000002182

In [214]:
map_cid2idx = pd.Series(data=range(len(unique_exist_cid)), index=unique_exist_cid)
map_cid2idx

CID000000085      0
CID000000137      1
CID000000143      2
CID000000158      3
CID000000159      4
               ... 
CID006398970    670
CID006436173    671
CID006447131    672
CID009571074    673
CID011947681    674
Length: 675, dtype: int64

In [235]:
df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist     = df_map_of_idx4ndc_rxcui_atc4_cids[df_map_of_idx4ndc_rxcui_atc4_cids.list_cid.notnull()].copy()
df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_not_exist = df_map_of_idx4ndc_rxcui_atc4_cids[df_map_of_idx4ndc_rxcui_atc4_cids.list_cid.isnull()].copy()

df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist

,NDC,RXCUI,idx,ATC4,list_cid
1,2.140701e+06,853004.0,174,C01BA,"[CID000004914, CID000004913, CID000003114, CID..."
4,2.322930e+06,352320.0,177,N06BA,"[CID000054841, CID000003007, CID000004723, CID..."
5,2.323560e+06,596928.0,178,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
6,2.324001e+06,596932.0,179,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
7,2.324030e+06,596932.0,180,N06AX,"[CID000005533, CID000004449, CID000060835, CID..."
...,...,...,...,...,...
4082,6.854601e+10,637218.0,4272,N04BD,[CID000122316]
4086,6.877401e+10,197516.0,4276,J01FA,"[CID000054688, CID003002190, CID000002269, CID..."
4088,6.885000e+10,995605.0,4278,J04AK,"[CID000003279, CID000001046]"
4089,6.885000e+10,995609.0,4279,J04AK,"[CID000003279, CID000001046]"


In order to speed up the calculation of DDI, the CID list needs to be converted into a CID index list in advance.

In [237]:
def list_cid2list_cid_idx(list_cid):
    return [map_cid2idx[cid] for cid in list_cid]

df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist['list_cid_idx'] = \
df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist['list_cid'].apply(list_cid2list_cid_idx)

df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist

,NDC,RXCUI,idx,ATC4,list_cid,list_cid_idx
1,2.140701e+06,853004.0,174,C01BA,"[CID000004914, CID000004913, CID000003114, CID...","[395, 394, 172, 25]"
4,2.322930e+06,352320.0,177,N06BA,"[CID000054841, CID000003007, CID000004723, CID...","[553, 160, 370, 311, 328, 27]"
5,2.323560e+06,596928.0,178,N06AX,"[CID000005533, CID000004449, CID000060835, CID...","[467, 339, 571, 325, 615, 26]"
6,2.324001e+06,596932.0,179,N06AX,"[CID000005533, CID000004449, CID000060835, CID...","[467, 339, 571, 325, 615, 26]"
7,2.324030e+06,596932.0,180,N06AX,"[CID000005533, CID000004449, CID000060835, CID...","[467, 339, 571, 325, 615, 26]"
...,...,...,...,...,...,...
4082,6.854601e+10,637218.0,4272,N04BD,[CID000122316],[610]
4086,6.877401e+10,197516.0,4276,J01FA,"[CID000054688, CID003002190, CID000002269, CID...","[551, 639, 69, 185]"
4088,6.885000e+10,995605.0,4278,J04AK,"[CID000003279, CID000001046]","[188, 24]"
4089,6.885000e+10,995609.0,4279,J04AK,"[CID000003279, CID000001046]","[188, 24]"


In [254]:
list(
    df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist.head().list_cid_idx.values
)

[[395, 394, 172, 25],
 [553, 160, 370, 311, 328, 27],
 [467, 339, 571, 325, 615, 26],
 [467, 339, 571, 325, 615, 26],
 [467, 339, 571, 325, 615, 26]]

In [239]:
df_map_of_idx4ndc_rxcui_atc4_cids = pd.concat([
    df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_not_exist,
    df_map_of_idx4ndc_rxcui_atc4_cids_list_cid_exist
])
df_map_of_idx4ndc_rxcui_atc4_cids

,NDC,RXCUI,idx,ATC4,list_cid,list_cid_idx
416,2.606320e+07,NaN,589,NaN,NaN,NaN
1351,1.320001e+08,NaN,1524,NaN,NaN,NaN
1798,3.380404e+08,NaN,1971,NaN,NaN,NaN
2569,8.290002e+09,NaN,2742,NaN,NaN,NaN
2576,8.881200e+09,NaN,2749,NaN,NaN,NaN
...,...,...,...,...,...,...
4082,6.854601e+10,637218.0,4272,N04BD,[CID000122316],[610]
4086,6.877401e+10,197516.0,4276,J01FA,"[CID000054688, CID003002190, CID000002269, CID...","[551, 639, 69, 185]"
4088,6.885000e+10,995605.0,4278,J04AK,"[CID000003279, CID000001046]","[188, 24]"
4089,6.885000e+10,995609.0,4279,J04AK,"[CID000003279, CID000001046]","[188, 24]"


In [240]:
df_map_of_idx4ndc_rxcui_atc4_cids.to_csv(os.path.join(path_ddi_dataset, "MAP_IDX4NDC_RXCUI_ATC4_CIDS.csv"))

## durg-DDI

In [69]:
df_drug_ddi = pd.read_csv(os.path.join(path_ddi_dataset, "drug-DDI.csv"))
df_drug_ddi

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache
...,...,...,...,...
4649436,CID000003461,CID000003954,C0149871,deep vein thromboses
4649437,CID000003461,CID000003954,C0035410,rhabdomyolysis
4649438,CID000003461,CID000003954,C0043096,loss of weight
4649439,CID000003461,CID000003954,C0003962,ascites


In [160]:
df_drug_ddi_exist_cid = df_drug_ddi[df_drug_ddi["STITCH 1"].isin(unique_exist_cid) &
                                    df_drug_ddi["STITCH 2"].isin(unique_exist_cid)]
df_drug_ddi_exist_cid

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
68,CID000003929,CID000150610,C0015230,eruption
69,CID000003929,CID000150610,C0001339,acute pancreatitis
70,CID000003929,CID000150610,C0032285,neumonia
71,CID000003929,CID000150610,C0013384,abnormal movements
72,CID000003929,CID000150610,C1145670,respiratory failure
...,...,...,...,...
4649436,CID000003461,CID000003954,C0149871,deep vein thromboses
4649437,CID000003461,CID000003954,C0035410,rhabdomyolysis
4649438,CID000003461,CID000003954,C0043096,loss of weight
4649439,CID000003461,CID000003954,C0003962,ascites


In [169]:
df_drug_ddi_exist_cid['idx_1'] = df_drug_ddi_exist_cid["STITCH 1"].map(map_cid2idx)
df_drug_ddi_exist_cid['idx_2'] = df_drug_ddi_exist_cid["STITCH 2"].map(map_cid2idx)
df_drug_ddi_exist_cid

/tmp/ipykernel_14242/3097511955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drug_ddi_exist_cid['idx_1'] = df_drug_ddi_exist_cid["STITCH 1"].map(map_cid2idx)
/tmp/ipykernel_14242/3097511955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drug_ddi_exist_cid['idx_2'] = df_drug_ddi_exist_cid["STITCH 2"].map(map_cid2idx)


,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name,idx_1,idx_2
68,CID000003929,CID000150610,C0015230,eruption,281,621
69,CID000003929,CID000150610,C0001339,acute pancreatitis,281,621
70,CID000003929,CID000150610,C0032285,neumonia,281,621
71,CID000003929,CID000150610,C0013384,abnormal movements,281,621
72,CID000003929,CID000150610,C1145670,respiratory failure,281,621
...,...,...,...,...,...,...
4649436,CID000003461,CID000003954,C0149871,deep vein thromboses,229,285
4649437,CID000003461,CID000003954,C0035410,rhabdomyolysis,229,285
4649438,CID000003461,CID000003954,C0043096,loss of weight,229,285
4649439,CID000003461,CID000003954,C0003962,ascites,229,285


👆👆👆

We can find that for the same drug pair, there may have multiple interactions!

In [177]:
df_drug_ddi_exist_cid.drop_duplicates(subset=["idx_1", "idx_2"])

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name,idx_1,idx_2
68,CID000003929,CID000150610,C0015230,eruption,281,621
469,CID000004946,CID000130881,C0038454,apoplexy,405,616
533,CID000005391,CID000005584,C0009806,constipated,439,476
569,CID000002802,CID005381226,C0008325,cholecystitis,141,663
684,CID000000596,CID000005726,C0013221,drug toxicity NOS,12,491
...,...,...,...,...,...,...
4648536,CID000002978,CID003468412,C0043096,loss of weight,155,642
4648780,CID000003961,CID000005212,C0017152,gastric inflammation,288,423
4649252,CID000002477,CID000002541,C0522055,abnormal ECG,91,99
4649335,CID000003750,CID000027661,C0017152,gastric inflammation,260,516


In [181]:
ddi_adj = np.zeros(
    (len(unique_exist_cid), len(unique_exist_cid))
)

for index, row in tqdm(df_drug_ddi_exist_cid.iterrows()):
    ddi_adj[row['idx_1']][row['idx_2']] += 1
    ddi_adj[row['idx_2']][row['idx_1']] += 1
    
ddi_adj

0it [00:00, ?it/s]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 8., 0., 0.],
       ...,
       [0., 0., 8., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [183]:
ddi_adj.max()

540.0

In [184]:
ddi_adj.mean()

13.395300960219478

In [185]:
ddi_adj.std()

38.724570061541826

In [186]:
ddi_adj.min()

0.0

In [249]:
with open(os.path.join(path_ddi_dataset, "ddi_adj_matrix.pickle"), 'wb') as f:
    pickle.dump(ddi_adj, f)

## DDI calculate

The interaction between the cid_idx(s) of same drug should be omitted!

In [4]:
df_map_of_idx4ndc_rxcui_atc4_cids = pd.read_csv(os.path.join(path_ddi_dataset, "MAP_IDX4NDC_RXCUI_ATC4_CIDS.csv"))
df_map_of_idx4ndc_rxcui_atc4_cids

,Unnamed: 0,NDC,RXCUI,idx,ATC4,list_cid,list_cid_idx
0,416,2.606320e+07,NaN,589,NaN,NaN,NaN
1,1351,1.320001e+08,NaN,1524,NaN,NaN,NaN
2,1798,3.380404e+08,NaN,1971,NaN,NaN,NaN
3,2569,8.290002e+09,NaN,2742,NaN,NaN,NaN
4,2576,8.881200e+09,NaN,2749,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4289,4082,6.854601e+10,637218.0,4272,N04BD,['CID000122316'],[610]
4290,4086,6.877401e+10,197516.0,4276,J01FA,"['CID000054688', 'CID003002190', 'CID000002269...","[551, 639, 69, 185]"
4291,4088,6.885000e+10,995605.0,4278,J04AK,"['CID000003279', 'CID000001046']","[188, 24]"
4292,4089,6.885000e+10,995609.0,4279,J04AK,"['CID000003279', 'CID000001046']","[188, 24]"


In [5]:
with open(os.path.join(path_ddi_dataset, "ddi_adj_matrix.pickle"), 'rb') as f:
    ddi_adj = pickle.load(f)
    
ddi_adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 8., 0., 0.],
       ...,
       [0., 0., 8., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
def calc_ddi_rate(pred_drug_ndc_scores_per_admi: torch.BoolTensor, 
                  df_map_of_idx4ndc_rxcui_atc4_cids, 
                  ddi_adj):
    df_map_of_idx4ndc_rxcui_atc4_cids.sort_values(by='idx', inplace=True)
    df_drugs_curr_admi = df_map_of_idx4ndc_rxcui_atc4_cids.loc[
        pred_drug_ndc_scores_per_admi.cpu().detach().numpy()
    ]
    df_drugs_can_calc_ddi = df_drugs_curr_admi[df_drugs_curr_admi.list_cid_idx.notnull()]
    
    q = Queue()
    for list_cid_idx in list(df_drugs_can_calc_ddi.list_cid_idx.values):
        q.put(list_cid_idx)
    
    cnt_all_pair = 0
    cnt_ddi_pair = 0
    while q.qsize() > 1:
        list_curr_cid_idx = q.get()
        for curr_cid_idx in eval(list_curr_cid_idx):
            for list_other_cid_idx in q.queue:
                for other_cid_idx in eval(list_other_cid_idx):
                    if (ddi_adj[curr_cid_idx][other_cid_idx] != 0):
                        cnt_ddi_pair += 1
                    cnt_all_pair += 1
                
    return 0 if cnt_all_pair == 0 else cnt_ddi_pair / cnt_all_pair

pred_drug_ndc_scores_per_admi = torch.BoolTensor([False] * 4294)
pred_drug_ndc_scores_per_admi[4279] = True
pred_drug_ndc_scores_per_admi[4292] = True

calc_ddi_rate(pred_drug_ndc_scores_per_admi, 
              df_map_of_idx4ndc_rxcui_atc4_cids, 
              ddi_adj)

0.16666666666666666

In [49]:
# [188, 24]
# [400, 169, 139]
ddi_adj[188][400], ddi_adj[188][169], ddi_adj[188][139]

(32.0, 0.0, 0.0)

In [50]:
ddi_adj[24][400], ddi_adj[24][169], ddi_adj[24][139]

(0.0, 0.0, 0.0)